In [74]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.impute import KNNImputer

In [75]:
df1 = pd.read_csv('COHORT_1_data.csv',usecols=['PATNO','ENROLL_AGE', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df2 = pd.read_csv('COHORT_2_data.csv',usecols=['PATNO','ENROLL_AGE', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df3 = pd.read_csv('COHORT_3_data.csv',usecols=['PATNO','ENROLL_AGE', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])
df4 = pd.read_csv('COHORT_4_data.csv',usecols=['PATNO','ENROLL_AGE', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT'])

In [76]:
# parkinson       ---->  1
# Healthy Cntrols ---->  2
# SWEDD           ---->  3
# Prodromal       ---->  4 

In [77]:
# Define a function to fill missing values and apply KNN imputation
def fill_missing_and_impute(df):
    # Group DataFrame by 'PATNO' and fill missing values with median within each group
    grouped_df = df.groupby('PATNO').transform(lambda x: x.fillna(x.median()))

    # Add 'PATNO' column back to the filled DataFrame
    df_filled = pd.concat([df['PATNO'], grouped_df], axis=1)

    # Calculate the mean of each feature after filling missing values
    grouped_mean_df = df_filled.groupby('PATNO').mean().reset_index()

    # Apply KNN imputation to handle remaining missing values
    knn = KNNImputer()
    grouped_mean_df = pd.DataFrame(knn.fit_transform(grouped_mean_df), columns=grouped_mean_df.columns)
    
    return grouped_mean_df

# Apply the function to all four dataframes
df1_filled = fill_missing_and_impute(df1)
df2_filled = fill_missing_and_impute(df2)
df3_filled = fill_missing_and_impute(df3)
df4_filled = fill_missing_and_impute(df4)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\A

In [78]:
# df1_filled.shape   --->  1752
# df2_filled.shape   --->  357
# df3_filled.shape   --->  81
# df4_filled.shape   ---> 1737

In [79]:
f = pd.concat([df1_filled,df2_filled,df3_filled,df4_filled])

In [111]:
sum_quip = ['QUIP_A','QUIP_B','QUIP_C','QUIP_D','QUIP_E']

f['QUIP_TOT'] = f[sum_quip].sum(axis=1)

In [114]:
sum_hvlt = ['HVLT_TOT_Recall', 'HVLT_DCR_REC','HVLT_RETENTION']
f['HVLT_TOT'] = f[sum_hvlt].sum(axis=1)

sum_gds = ['GDS_TOT', 'GDS_Depressed']
f['NGDS_TOT'] = f[sum_gds].sum(axis=1)

sum_ess = ['ESS_TOT', 'ESS_Sleepy']
f['NESS_TOT'] = f[sum_ess].sum(axis=1)

In [115]:
f.columns

Index(['PATNO', 'SDMTOTAL', 'STAI_TOT', 'SFT_TOT', 'SCOPA_AUT_TOT',
       'REMSLEEP_TOT', 'QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
       'MoCA_score', 'LNS_TOT', 'HVLT_TOT_Recall', 'HVLT_DCR_REC',
       'HVLT_RETENTION', 'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy',
       'BJLOT_TOT', 'COHORT', 'ENROLL_AGE', 'QUIP_TOT', 'HVLT_TOT', 'NGDS_TOT',
       'NESS_TOT'],
      dtype='object')

In [116]:
# Calculate the correlation matrix
correlation_matrix = f.corr()

# Get the correlation of 'cohort' with all other categories
cohort_correlation = correlation_matrix['COHORT']

print(cohort_correlation)

PATNO              0.167667
SDMTOTAL           0.179086
STAI_TOT          -0.131191
SFT_TOT           -0.043927
SCOPA_AUT_TOT     -0.079901
REMSLEEP_TOT       0.093612
QUIP_A            -0.034002
QUIP_B            -0.021331
QUIP_C             0.011175
QUIP_D             0.070924
QUIP_E             0.012671
MoCA_score         0.027158
LNS_TOT            0.094343
HVLT_TOT_Recall    0.047585
HVLT_DCR_REC       0.036601
HVLT_RETENTION     0.013834
GDS_TOT           -0.129484
GDS_Depressed     -0.092210
ESS_TOT           -0.074476
ESS_Sleepy        -0.078770
BJLOT_TOT          0.069151
COHORT             1.000000
ENROLL_AGE         0.196641
QUIP_TOT           0.028439
HVLT_TOT           0.048521
NGDS_TOT          -0.127065
NESS_TOT          -0.075870
Name: COHORT, dtype: float64


In [81]:
le = LabelEncoder()
f['COHORT'] = le.fit_transform(f['COHORT'])

In [118]:
X =  f.drop(['COHORT','PATNO','QUIP_A', 'QUIP_B', 'QUIP_C', 'QUIP_D', 'QUIP_E',
             'HVLT_TOT_Recall', 'HVLT_DCR_REC','HVLT_RETENTION',
             'GDS_TOT', 'GDS_Depressed', 'ESS_TOT', 'ESS_Sleepy'],axis=1)
y= f['COHORT']



In [119]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import make_pipeline

In [120]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [85]:
# adasyn = ADASYN(sampling_strategy='auto',random_state=42)
# X_adasyn, y_adasyn = adasyn.fit_resample(X, y)

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X_smote,y_smote, 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True)

In [122]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [123]:
svm = SVC(kernel="rbf", gamma="auto",probability=True)
svm.fit(X_train,y_train)
prediction = svm.predict(X_test)

accuracy_SVM = accuracy_score(prediction, y_test)
print(accuracy_SVM)

0.7351598173515982


In [124]:
dec = DecisionTreeClassifier(random_state=0)
dec.fit(X_train,y_train)
dec_pred = dec.predict(X_test)
accuracy_DEC = accuracy_score(dec_pred, y_test)
print(accuracy_DEC)

0.7248858447488584


In [126]:
rand = RandomForestClassifier(max_depth=15, random_state=0)
rand.fit(X_train,y_train)
rand_pred = rand.predict(X_test)
rand_accuracy = accuracy_score(rand_pred,y_test)
print(rand_accuracy)

0.8127853881278538


In [135]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train,y_train)
neigh_pred = neigh.predict(X_test)
neigh_accuracy = accuracy_score(neigh_pred,y_test)
print(neigh_accuracy)

0.735730593607306


In [128]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, and y_test are already defined

# Initialize and train the gradient boosting classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)

# Make predictions on the test set
gb_pred = gb_clf.predict(X_test)

# Calculate accuracy
gb_accuracy = accuracy_score(gb_pred, y_test)
print("Gradient Boosting Classifier Accuracy:", gb_accuracy)

Gradient Boosting Classifier Accuracy: 0.757420091324201


In [129]:
from sklearn.ensemble import VotingClassifier
# Create a Soft Voting Classifier
labels = ['Decision Forest', 'Random Forest', 'GB','KNN']
voting_clf_hard = VotingClassifier(
	estimators=[
        (labels[0], dec),
		(labels[1], rand), # Include the second classifier 
		(labels[2],gb_clf),
        # (labels[3], neigh), # Include the third classifier 
	],
	voting='hard' # Specify soft voting, where class probabilities are combined
)


In [130]:
voting_clf_hard.fit(X_train,y_train)
V_hard_pred = voting_clf_hard.predict(X_test)
voting_hard_accuracy = accuracy_score(V_hard_pred,y_test)
print(voting_hard_accuracy)

0.809931506849315


In [131]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
cross_val_results = cross_val_score(voting_clf_hard, X, y, cv=kf)
print(f'Cross-Validation Results (Accuracy): {cross_val_results}')
print(f'Mean Accuracy: {cross_val_results.mean()}')

Cross-Validation Results (Accuracy): [0.69338422 0.67938931 0.67643312 0.68535032 0.70828025]
Mean Accuracy: 0.688567446232638
